In [1]:
import re
import pandas as pd
import matplotlib.pyplot as plt
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/katiegarrett/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [2]:
top_2019 = pd.read_csv('../data/top_2019.csv')
top_2019.head()

,Unnamed: 0,author,author_flair_css_class,author_flair_richtext,author_flair_text,author_flair_type,author_fullname,author_patreon_flair,can_mod_post,contest_mode,...,thumbnail_width,author_cakeday,media_metadata,crosspost_parent,crosspost_parent_list,media,media_embed,secure_media,secure_media_embed,date
0,0,Son_of_York,NaN,[],NaN,text,t2_45if3,False,False,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019-03-01 16:32:02+00:00
1,1,P_Ell_Travers,NaN,[],NaN,text,t2_3b5yhdh,False,False,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019-02-16 12:56:47+00:00
2,2,theodore_boozevelt,NaN,"[{'e': 'text', 't': 'High School French '}]",High School French,richtext,t2_cja60,False,False,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019-03-29 23:08:28+00:00
3,3,content404,NaN,[],NaN,text,t2_6dnx6,False,False,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019-05-03 20:08:03+00:00
4,4,lordofthewastelands,NaN,[],NaN,text,t2_31ahu5di,False,False,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019-03-02 12:54:59+00:00


In [3]:
top_2019["combined"] = top_2019["title"].astype(str) + top_2019["selftext"].astype(str)

In [5]:
top_2019["row_id"] = top_2019.index + 1 
print (top_2019.head(10))

   Unnamed: 0               author  author_flair_css_class  \
0           0          Son_of_York                     NaN   
1           1        P_Ell_Travers                     NaN   
2           2   theodore_boozevelt                     NaN   
3           3           content404                     NaN   
4           4  lordofthewastelands                     NaN   
5           5             A2I_WWJD                     NaN   
6           6          imnotwarren                     NaN   
7           7       The_Gr8_Catsby                     NaN   
8           8         navychic7600                     NaN   
9           9            mdawg1279                     NaN   

                         author_flair_richtext    author_flair_text  \
0                                           []                  NaN   
1                                           []                  NaN   
2  [{'e': 'text', 't': 'High School French '}]  High School French    
3                                

In [6]:
top_2019_subset = top_2019[['row_id', 'combined']].copy()
#data clean-up
#remove all non-alphabet characters
top_2019_subset['combined'] = top_2019_subset['combined'].str.replace("[^a-zA-Z#]", " ")
#convert to lower-case
top_2019_subset['combined'] = top_2019_subset['combined'].str.casefold()
print (top_2019_subset.head(10))

   row_id                                           combined
0       1  i m calling it  we have lost the classroom arm...
1       2  apparently teaching girls to have self esteem ...
2       3   warning  politics  devos believes that by red...
3       4  today they used mia khalifa and riley reid as ...
4       5  i m tired of news articles spotlighting teache...
5       6  us state dept  of ed s should start reporting ...
6       7  my shyest student just gave a    minute presen...
7       8  i may not be the best at classroom management ...
8       9  what if we did all of this while he was alive ...
9      10  best sub everthe sub broke up a fight   took d...


/var/folders/4_/948td82x7jlf4pz0c4dtqn_00000gn/T/ipykernel_31388/508247667.py:4: FutureWarning: The default value of regex will change from True to False in a future version.
  top_2019_subset['combined'] = top_2019_subset['combined'].str.replace("[^a-zA-Z#]", " ")


In [7]:
df1=pd.DataFrame()
df1['row_id']=['99999999999']
df1['sentiment_type']='NA999NA'
df1['sentiment_score']=0

In [8]:
print('Processing sentiment analysis...')
sid = SentimentIntensityAnalyzer()
t_df = df1
for index, row in top_2019_subset.iterrows():
    scores = sid.polarity_scores(row[1])
    for key, value in scores.items():
        temp = [key,value,row[0]]
        df1['row_id']=row[0]
        df1['sentiment_type']=key
        df1['sentiment_score']=value
        t_df=t_df.append(df1)
#remove dummy row with row_id = 99999999999
t_df_cleaned = t_df[t_df.row_id != '99999999999']
#remove duplicates if any exist
t_df_cleaned = t_df_cleaned.drop_duplicates()
# only keep rows where sentiment_type = compound
t_df_cleaned = t_df[t_df.sentiment_type == 'compound']
print(t_df_cleaned.head(10))

Processing sentiment analysis...
   row_id sentiment_type  sentiment_score
0       1       compound           0.8519
0       2       compound           0.9825
0       3       compound          -0.1087
0       4       compound          -0.0772
0       5       compound          -0.4404
0       6       compound           0.0000
0       7       compound          -0.2819
0       8       compound          -0.8935
0       9       compound           0.9790
0      10       compound           0.8948


In [9]:
df_output = pd.merge(top_2019, t_df_cleaned, on='row_id', how='inner')
print(df_output.head(10))

   Unnamed: 0               author  author_flair_css_class  \
0           0          Son_of_York                     NaN   
1           1        P_Ell_Travers                     NaN   
2           2   theodore_boozevelt                     NaN   
3           3           content404                     NaN   
4           4  lordofthewastelands                     NaN   
5           5             A2I_WWJD                     NaN   
6           6          imnotwarren                     NaN   
7           7       The_Gr8_Catsby                     NaN   
8           8         navychic7600                     NaN   
9           9            mdawg1279                     NaN   

                         author_flair_richtext    author_flair_text  \
0                                           []                  NaN   
1                                           []                  NaN   
2  [{'e': 'text', 't': 'High School French '}]  High School French    
3                                

In [10]:
df_output[["sentiment_score"]].describe()

,sentiment_score
count,670.000000
mean,0.276179
std,0.690181
min,-0.999000
25%,-0.291875
50%,0.549900
75%,0.885775
max,0.999800


In [11]:
top_2019_final = df_output

In [12]:
top_2019_final['year'] = 2019

In [14]:
top_2019_final.head()

,Unnamed: 0,author,author_flair_css_class,author_flair_richtext,author_flair_text,author_flair_type,author_fullname,author_patreon_flair,can_mod_post,contest_mode,...,media,media_embed,secure_media,secure_media_embed,date,combined,row_id,sentiment_type,sentiment_score,year
0,0,Son_of_York,NaN,[],NaN,text,t2_45if3,False,False,False,...,NaN,NaN,NaN,NaN,2019-03-01 16:32:02+00:00,I'm calling it. We have lost the classroom arm...,1,compound,0.8519,2019
1,1,P_Ell_Travers,NaN,[],NaN,text,t2_3b5yhdh,False,False,False,...,NaN,NaN,NaN,NaN,2019-02-16 12:56:47+00:00,Apparently teaching girls to have self esteem ...,2,compound,0.9825,2019
2,2,theodore_boozevelt,NaN,"[{'e': 'text', 't': 'High School French '}]",High School French,richtext,t2_cja60,False,False,False,...,NaN,NaN,NaN,NaN,2019-03-29 23:08:28+00:00,[Warning: Politics] DeVos believes that by red...,3,compound,-0.1087,2019
3,3,content404,NaN,[],NaN,text,t2_6dnx6,False,False,False,...,NaN,NaN,NaN,NaN,2019-05-03 20:08:03+00:00,Today they used mia khalifa and riley reid as ...,4,compound,-0.0772,2019
4,4,lordofthewastelands,NaN,[],NaN,text,t2_31ahu5di,False,False,False,...,NaN,NaN,NaN,NaN,2019-03-02 12:54:59+00:00,I’m tired of news articles spotlighting teache...,5,compound,-0.4404,2019


In [15]:
top_2019_final.to_csv('top_2019_final.csv')